#FNC

## Dataset Loding and Splitting

In [ ]:
import pandas as pd
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Input, Dropout, MaxPooling1D, Convolution1D
from keras.layers import LSTM, Lambda, merge, Masking
from keras.layers import Embedding, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
import numpy as np
import tensorflow as tf
import re
from keras import backend as K
import keras.callbacks
import sys
import os
import theano as theano
import theano.tensor as T
from theano.gradient import grad_clip
import time
import operator
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from gensim.models import Word2Vec
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Mount data from drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path="/content/drive/MyDrive/IIITH- Folder/SMAI/SMAI_Project/FNC dataset"

In [ ]:
train_body = pd.read_csv("/content/drive/MyDrive/IIITH- Folder/SMAI/SMAI_Project/FNC dataset/train_bodies.csv")
train_stances = pd.read_csv("/content/drive/MyDrive/IIITH- Folder/SMAI/SMAI_Project/FNC dataset/train_stances.csv")
test_body = pd.read_csv("/content/drive/MyDrive/IIITH- Folder/SMAI/SMAI_Project/FNC dataset/competition_test_bodies.csv")
test_stances = pd.read_csv("/content/drive/MyDrive/IIITH- Folder/SMAI/SMAI_Project/FNC dataset/competition_test_stances.csv")

In [ ]:
train_body
train_body_copy = train_body.copy()
train_body_col = train_body.columns
train_body_col

Index(['Body ID', 'articleBody'], dtype='object')

In [ ]:
train_stances
train_stances_copy = train_stances.copy()
train_stances_col = train_stances.columns
train_stances_col

Index(['Headline', 'Body ID', 'Stance'], dtype='object')

In [ ]:
print(train_stances.shape)
print(train_body.shape)

(49972, 3)
(1683, 2)


In [ ]:
stances_count = train_stances['Body ID'].unique()
body_count = train_body['Body ID'].unique()

In [ ]:
count = 0
for i in stances_count:
    if i not in body_count:
        count += 1

count

0

In [ ]:
body_stance_merged = pd.merge(train_body_copy, train_stances_copy, on='Body ID')

In [ ]:
body_stance_merged.shape

(49972, 4)

In [ ]:
test_body_copy = test_body
test_stances_copy = test_stances

In [ ]:
test_body_copy.head(5)


,Body ID,articleBody
0,1,Al-Sisi has denied Israeli reports stating tha...
1,2,A bereaved Afghan mother took revenge on the T...
2,3,CNBC is reporting Tesla has chosen Nevada as t...
3,12,A 4-inch version of the iPhone 6 is said to be...
4,19,GR editor’s Note\n\nThere are no reports in th...


In [ ]:
test_stances_copy.head(5)

,Headline,Body ID,Stance
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,unrelated
4,Argentina's President Adopts Boy to End Werewo...,37,unrelated


In [ ]:
body_stance_merged

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated
...,...,...,...,...
49967,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Pizza delivery man gets tipped more than $2,00...",agree
49968,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Pizza delivery man gets $2,000 tip",agree
49969,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Luckiest Pizza Delivery Guy Ever Gets $2,000 Tip",agree
49970,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...",Ann Arbor pizza delivery driver surprised with...,agree


In [ ]:
train_body_stance_merged_copy, val_body_stance_merged_copy = train_test_split(body_stance_merged,test_size = 0.70)

In [ ]:
val_body_stance_merged_copy, remaining_data = train_test_split(body_stance_merged,test_size = 0.70)

In [ ]:
train_body_stance_merged_copy.shape

(14991, 4)

In [ ]:
val_body_stance_merged_copy.shape

(14991, 4)

In [ ]:
train_body_stance_merged = train_body_stance_merged_copy

In [ ]:
val_body_stance_merged = val_body_stance_merged_copy

In [ ]:
train_body_copy_2 = train_body_copy.copy()


In [ ]:
test_body_stance_merged = pd.merge(test_body_copy, test_stances_copy, on='Body ID')

In [ ]:
test_body_stance_merged.shape

(25413, 4)

In [ ]:
"""
test_body_stance_merged
train_body_stance_merged
val_body_stance_merged
"""

'\ntest_body_stance_merged\ntrain_body_stance_merged\nval_body_stance_merged\n'

In [ ]:
test_body_stance_merged.head(5)

,Body ID,articleBody,Headline,Stance
0,1,Al-Sisi has denied Israeli reports stating tha...,Apple installing safes in-store to protect gol...,unrelated
1,1,Al-Sisi has denied Israeli reports stating tha...,El-Sisi denies claims he'll give Sinai land to...,agree
2,1,Al-Sisi has denied Israeli reports stating tha...,Apple to keep gold Watch Editions in special i...,unrelated
3,1,Al-Sisi has denied Israeli reports stating tha...,Apple Stores to Keep Gold “Edition” Apple Watc...,unrelated
4,1,Al-Sisi has denied Israeli reports stating tha...,South Korean woman's hair 'eaten' by robot vac...,unrelated


In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import simple_preprocess
from gensim.parsing.porter import PorterStemmer
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
import time

##Converting Stance to Categorial

In [ ]:
def convert_categorical_data(dataset):
  stance_dictionary = {'agree':0 , 'discuss':0 , 'unrelated':1, 'disagree':1}
  tmp = dataset['Stance'].map(stance_dictionary)
  return tmp



In [ ]:
  train_body_stance_merged['Stance'] = convert_categorical_data(train_body_stance_merged)
  val_body_stance_merged['Stance'] = convert_categorical_data(val_body_stance_merged)
  test_body_stance_merged['Stance'] =convert_categorical_data(test_body_stance_merged)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
def check_for_nulls(dataset):
  return dataset['articleBody'].isnull().values.any()


In [ ]:
check_for_nulls(train_body_stance_merged)

False

In [ ]:
val_body_stance_merged['Headline'].isnull().values.any()

False

Steps
1. wordToVec
2. Convert both Headline and articleBody to vector.
3. get hidden vector from BiGRU
4. Pass both vectors  to dense layers parallely and  find cosine similarities btw two
5. Classify

In [ ]:
train_articleBody = train_body_stance_merged["articleBody"]
val_articleBody = val_body_stance_merged["articleBody"]
test_articleBody = test_body_stance_merged["articleBody"]

In [ ]:
train_Headline = train_body_stance_merged["Headline"]
val_Headline = val_body_stance_merged["Headline"]
test_Headline = test_body_stance_merged["Headline"]

COPY

In [ ]:
train_articleBodycp=train_articleBody.copy()
train_Headlinecp=train_Headline.copy()

##wordToVec

In [ ]:
def preprocessing(dataset):
  # remove stopwords
  dataset = dataset.apply(remove_stopwords)
  # preprocess
  dataset = [simple_preprocess(line, deacc=True) for line in dataset]
  porter_stemmer = PorterStemmer()
  # stemmed_tokens
  tmp = [[porter_stemmer.stem(word) for word in tokens] for tokens in dataset]
  return tmp


In [ ]:
train_Headlinecp = preprocessing(train_Headlinecp)
train_articleBodycp = preprocessing(train_articleBodycp)

In [ ]:
print(type(train_Headlinecp))

<class 'list'>


train

In [ ]:
def train_wordToVec(dataset, filename):
  size = 300
  window = 3
  min_count = 1
  workers = 3
  sg = 1 
  OUTPUT_FOLDER = "/content/drive/MyDrive/"
  word2vec_model_file1 = OUTPUT_FOLDER + filename + str(size) + '.model'
  start_time = time.time()
  stemmed_tokens1 = pd.Series(dataset).values
  # Train the Word2Vec Model
  w2v_model1 = Word2Vec(stemmed_tokens1, min_count = min_count, size = size, workers = workers, window = window, sg = sg)
  print("Time taken to train word2vec model: " + str(time.time() - start_time))
  w2v_model1.save(word2vec_model_file1)

  return stemmed_tokens1, word2vec_model_file1


In [ ]:
stemmed_tokens1, word2vec_model_file1 = train_wordToVec(train_Headlinecp,"word2vec_1")
stemmed_tokens2, word2vec_model_file2 = train_wordToVec(train_articleBodycp,"word2vec_2" )

Time taken to train word2vec model: 2.7957489490509033
Time taken to train word2vec model: 76.21452975273132


In [ ]:
w2v_model1= Word2Vec.load(word2vec_model_file1)
w2v_model2= Word2Vec.load(word2vec_model_file2)

In [ ]:
w2v_model1.wv.save_word2vec_format('myfile1.txt',binary=False)
w2v_model2.wv.save_word2vec_format('myfile2.txt',binary=False)

##Embedding Matrix of the Training data and Creating Input tensor

In [ ]:
import os
def get_embed_index(filename):
  embed_index1={}
  f= open(os.path.join('',filename),encoding="utf-8")
  for line in f:
    values =line.split()
    word = values[0]
    coefs= np.asarray(values[1:])
    embed_index1[word]=coefs
  f.close()
  return embed_index1

In [ ]:
embed_index1=get_embed_index('myfile1.txt')
embed_index2=get_embed_index('myfile2.txt')

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [ ]:
def get_embed_matrix(stemmed_tokens, embed_index,type):
  tok_obj=Tokenizer()
  tok_obj.fit_on_texts(stemmed_tokens)
  sequences = tok_obj.texts_to_sequences(stemmed_tokens)

  word_index=tok_obj.word_index
  print('Found %s unique tokens.'% len(word_index))

  if type=='H':
    #max_length=max([len(s) for s in stemmed_tokens])
    max_length=25
  else:
    max_length=1500

  review_pad =pad_sequences(sequences,maxlen=max_length)

  print("Shape of review tensor:",review_pad.shape)
  num_words=len(word_index)+1
  embed_matrix=np.zeros((num_words,300))

  for word,i in word_index.items():
    if i > num_words:
      continue
    embedding_vector=embed_index.get(word)
    if embedding_vector is not None:
      embed_matrix[i]=embedding_vector
  return embed_matrix,num_words,max_length, review_pad


###Embedding Matrix and Tensor of Train data

In [ ]:
embed_matrix1,num_words1,max_length1,tensor1=get_embed_matrix(stemmed_tokens1,embed_index1,'H')
embed_matrix2,num_words2,max_length2, tensor2=get_embed_matrix(stemmed_tokens2, embed_index2,'A')

Found 2554 unique tokens.
Shape of review tensor: (14991, 25)
Found 14697 unique tokens.
Shape of review tensor: (14991, 1500)


In [ ]:
tensor2.shape

(14991, 1500)

In [ ]:
print(embed_matrix1.shape)

(2555, 300)


In [ ]:
print(embed_matrix2.shape)

(14698, 300)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, SimpleRNN, Concatenate, Flatten
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.layers import Bidirectional, GlobalMaxPool1D

In [ ]:
train_body_stance_merged['Stance']

12375    1
38627    0
32812    1
18713    1
24501    1
        ..
18295    1
12267    1
36354    1
14671    1
42101    0
Name: Stance, Length: 14991, dtype: int64

In [ ]:
train_body_stance_merged.shape

(14991, 4)

In [ ]:
# print(len(out_list))

In [ ]:
# model.fit(inp_list, out_list.astype("float64"), batch_size = 64,epochs=2)

##Baseline Model

In [ ]:
#Embedding Input
embed1=Input(shape=(25,))
embed2=Input(shape=(1500,))

#Embedding Layer
embedding_layer1 = Embedding( num_words1,300,embeddings_initializer=Constant(embed_matrix1),input_length=max_length1,trainable=False)(embed1)
embedding_layer2 = Embedding( num_words2,300,embeddings_initializer=Constant(embed_matrix2),input_length=max_length2,trainable=False)(embed2)
#Bigru layer
bigrulayer1=Bidirectional(GRU(units=250, dropout=0.5, recurrent_dropout=0.2))(embedding_layer1)
bigrulayer2=Bidirectional(GRU(units=250, dropout=0.5, recurrent_dropout=0.2))(embedding_layer2)
#Similarity layer
pin_part = Dense(64, activation='sigmoid')(bigrulayer1)
sku_part = Dense(64, activation='sigmoid')(bigrulayer2)
#Cosine global similarity
global_similarity=keras.layers.Dot(axes=1, normalize=True)([pin_part, sku_part])
#Prediction
probb= Dense(1, activation='softmax')(global_similarity)

model1 = Model(inputs=[embed1] + [embed2], outputs=probb)
model1.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1500)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 25, 300)      766500      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1500, 300)    4409400     input_2[0][0]                    
______________________________________________________________________________________________

### Input and Output

In [ ]:
inp_list = [tensor1, tensor2]
out_list = train_body_stance_merged['Stance']

### Fitting the Model

In [ ]:
model1.fit(inp_list, out_list.astype("float64"), batch_size = 64,epochs=2)

Epoch 1/2
235/235 [==============================] - 6443s 27s/step - loss: 0.7067 - accuracy: 0.7439
Epoch 2/2
235/235 [==============================] - 6723s 29s/step - loss: 0.6327 - accuracy: 0.7462


##Validation Data 

In [ ]:
val_articleBodycp=val_articleBody.copy()
val_Headlinecp=val_Headline.copy()

In [ ]:
val_Headlinecp = preprocessing(val_Headlinecp)
val_articleBodycp = preprocessing(val_articleBodycp)

In [ ]:
stemmed_tokens_val1, word2vec_model_file_v1 = train_wordToVec(val_Headlinecp,"word2vec_v1")
stemmed_tokens_val2, word2vec_model_file_v2 = train_wordToVec(val_articleBodycp,"word2vec_v2" )

Time taken to train word2vec model: 3.0138747692108154
Time taken to train word2vec model: 89.79340839385986


In [ ]:
w2v_model_v1= Word2Vec.load(word2vec_model_file_v1)
w2v_model_v2= Word2Vec.load(word2vec_model_file_v2)

In [ ]:
w2v_model1.wv.save_word2vec_format('myfile_v1.txt',binary=False)
w2v_model2.wv.save_word2vec_format('myfile_v2.txt',binary=False)

In [ ]:
embed_index_v1=get_embed_index('myfile_v1.txt')
embed_index_v2=get_embed_index('myfile_v2.txt')

In [ ]:
embed_matrix_v1,num_words_v1,max_length_v1,tensor_v1=get_embed_matrix(stemmed_tokens_val1,embed_index_v1,'H')
embed_matrix_v2,num_words_v2,max_length_v2, tensor_v2=get_embed_matrix(stemmed_tokens_val2, embed_index_v2,'A')

Found 2552 unique tokens.
Shape of review tensor: (14991, 25)
Found 14688 unique tokens.
Shape of review tensor: (14991, 1500)




---



##Testing Data

In [ ]:
test_articleBodycp=test_articleBody.copy()
test_Headlinecp=test_Headline.copy()

In [ ]:
test_Headlinecp = preprocessing(test_Headlinecp)
test_articleBodycp = preprocessing(test_articleBodycp)

In [ ]:
stemmed_tokenst1, word2vec_model_filet1 = train_wordToVec(test_Headlinecp[:10000],"word2vec_t1")
stemmed_tokenst2, word2vec_model_filet2 = train_wordToVec(test_articleBodycp[:10000],"word2vec_t2" )

Time taken to train word2vec model: 1.9092988967895508
Time taken to train word2vec model: 46.1835412979126


In [ ]:
w2v_modelt1= Word2Vec.load(word2vec_model_filet1)
w2v_modelt2= Word2Vec.load(word2vec_model_filet1)

In [ ]:
w2v_model1.wv.save_word2vec_format('myfilet1.txt',binary=False)
w2v_model2.wv.save_word2vec_format('myfilet2.txt',binary=False)

In [ ]:
embed_indext1=get_embed_index('myfilet1.txt')
embed_indext2=get_embed_index('myfilet2.txt')

In [ ]:
embed_matrixt1,num_wordst1,max_lengtht1,tensort1=get_embed_matrix(stemmed_tokenst1,embed_indext1,'H')
embed_matrixt2,num_wordst2,max_lengtht2, tensort2=get_embed_matrix(stemmed_tokenst2, embed_indext2,'A')

Found 1695 unique tokens.
Shape of review tensor: (10000, 25)
Found 7633 unique tokens.
Shape of review tensor: (10000, 1500)


In [ ]:
out_list = test_body_stance_merged['Stance'][:10000]

In [ ]:
out_listl=out_list.tolist()

### Test Accuracy

In [ ]:
petruth_means = model.predict([tensort1,tensort2])

In [ ]:
tetruthClass = []
predclass = []
from sklearn import metrics

for i in range(len(petruth_means)):
  
    if petruth_means[i] >= 0.5:
        predclass.append(0)
    else:
        predclass.append(1)

accuracy = metrics.accuracy_score(out_listl,predclass)


In [ ]:
score = model1.evaluate([tensort1,tensort2], out_list, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.5669732689857483 / Test accuracy: 0.8346999883651733


##Experiments

###Experiment1 - Trainset- 15k and Testset- 10k

---



In [ ]:
embed1=Input(shape=(25,))
embed2=Input(shape=(1500,))
 
embedding_layer1 = Embedding( num_words1,300,embeddings_initializer=Constant(embed_matrix1),
                             input_length=max_length1,trainable=False)(embed1)
embedding_layer2 = Embedding( num_words2,300,embeddings_initializer=Constant(embed_matrix2),
                             input_length=max_length2,trainable=False)(embed2)
bigrulayer1=Bidirectional(GRU(units=250, dropout=0.3, recurrent_dropout=0.5))(embedding_layer1)
bigrulayer2=Bidirectional(GRU(units=250, dropout=0.3, recurrent_dropout=0.5))(embedding_layer2)
 
pin_part = Dense(64, activation='relu')(bigrulayer1)
sku_part = Dense(64, activation='relu')(bigrulayer2)

global_similarity=keras.layers.Dot(axes=1, normalize=True)([pin_part, sku_part])

probb= Dense(1, activation='softmax')(global_similarity)

model1 = Model(inputs=[embed1] + [embed2], outputs=probb)
model1.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1500)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 25, 300)      763500      input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1500, 300)    4429200     input_4[0][0]                    
____________________________________________________________________________________________

In [ ]:
model1.fit(inp_list, out_list.astype("float64"), batch_size = 64,epochs=2)

Epoch 1/2
235/235 [==============================] - 7324s 31s/step - loss: 0.6869 - accuracy: 0.7448
Epoch 2/2
235/235 [==============================] - 7342s 31s/step - loss: 0.6382 - accuracy: 0.7442


In [ ]:
score = model1.evaluate([tensort1,tensort2], out_list, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.5790790319442749 / Test accuracy: 0.8346999883651733


###Experiment2 - Trainset- 15k and Testset- 10k

---

In [ ]:
embed1=Input(shape=(25,))
embed2=Input(shape=(1500,))
 
embedding_layer1 = Embedding( num_words1,300,embeddings_initializer=Constant(embed_matrix1),input_length=max_length1,trainable=False)(embed1)
embedding_layer2 = Embedding( num_words2,300,embeddings_initializer=Constant(embed_matrix2),input_length=max_length2,trainable=False)(embed2)
bigrulayer1=Bidirectional(GRU(units=100, dropout=0.5, recurrent_dropout=0.2))(embedding_layer1)
bigrulayer2=Bidirectional(GRU(units=100, dropout=0.5, recurrent_dropout=0.2))(embedding_layer2)
 
pin_part = Dense(128, activation='relu')(bigrulayer1)
pin_part_1 = Dropout(0.2)(pin_part)
pin_part_2 = Dense(64, activation='relu')(pin_part_1)
sk_part = Dense(128, activation='relu')(bigrulayer2)
sk_part_1 = Dropout(0.2)(sk_part)
sk_part_2 = Dense(64, activation='relu')(sk_part_1)

global_similarity=keras.layers.Dot(axes=1, normalize=True)([pin_part_2, sk_part_2])

probb= Dense(1, activation='softmax')(global_similarity)

model1 = Model(inputs=[embed1] + [embed2], outputs=probb)
model1.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 1500)]       0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 25, 300)      767100      input_13[0][0]                   
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 1500, 300)    4406100     input_14[0][0]                   
____________________________________________________________________________________________

In [ ]:
model1.fit(inp_list, out_list.astype("float64"), batch_size = 64,epochs=5)

Epoch 1/5
235/235 [==============================] - 3103s 13s/step - loss: 0.6861 - accuracy: 0.7565
Epoch 2/5
235/235 [==============================] - 3134s 13s/step - loss: 0.6358 - accuracy: 0.7543
Epoch 3/5
235/235 [==============================] - 3246s 14s/step - loss: 0.6076 - accuracy: 0.7483
Epoch 4/5
235/235 [==============================] - 3206s 14s/step - loss: 0.5882 - accuracy: 0.7493
Epoch 5/5
235/235 [==============================] - 3269s 14s/step - loss: 0.5744 - accuracy: 0.7522




---



Testing

In [ ]:
score = model1.evaluate([tensort1,tensort2], out_list, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.5012915730476379 / Test accuracy: 0.8356666564941406


###Experiment3 - On small dataset

In [ ]:
embed1=Input(shape=(25,))
embed2=Input(shape=(1500,))
 
embedding_layer1 = Embedding( num_words1,300,embeddings_initializer=Constant(embed_matrix1),input_length=max_length1,trainable=False)(embed1)
embedding_layer2 = Embedding( num_words2,300,embeddings_initializer=Constant(embed_matrix2),input_length=max_length2,trainable=False)(embed2)
bigrulayer1=Bidirectional(GRU(units=250, dropout=0.3, recurrent_dropout=0.5))(embedding_layer1)
bigrulayer2=Bidirectional(GRU(units=250, dropout=0.3, recurrent_dropout=0.5))(embedding_layer2)
 
pin_part = Dense(64, activation='relu')(bigrulayer1)
sku_part = Dense(64, activation='relu')(bigrulayer2)

global_similarity=keras.layers.Dot(axes=1, normalize=True)([pin_part, sku_part])

probb= Dense(1, activation='softmax')(global_similarity)

model1 = Model(inputs=[embed1] + [embed2], outputs=probb)
model1.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1500)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 25, 300)      513900      input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1500, 300)    3093900     input_4[0][0]                    
____________________________________________________________________________________________

In [ ]:
model1.fit(inp_list, out_list.astype("float64"), batch_size = 64,epochs=2)

Epoch 1/2
16/16 [==============================] - 524s 32s/step - loss: 0.5971 - accuracy: 0.7701
Epoch 2/2
16/16 [==============================] - 516s 32s/step - loss: 0.5907 - accuracy: 0.7715


In [ ]:
score = model1.evaluate([tensort1,tensort2], out_list, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.4944207966327667 / Test accuracy: 0.9539999961853027


# Clickbait

# CLICKBAIT DATASET



  ### Fields in instances.jsonl:
  { 

    "id": "<instance id>",
    "postTimestamp": "<weekday> <month> <day> <hour>:<minute>:<second> <time_offset> <year>",
    "postText": ["<text of the post with links removed>"],
    "postMedia": ["<path to a file in the media archive>"],
    "targetTitle": "<title of target article>",
    "targetDescription": "<description tag of target article>",
    "targetKeywords": "<keywords tag of target article>",
    "targetParagraphs": ["<text of the ith paragraph in the target article>"],
    "targetCaptions": ["<caption of the ith image in the target article>"]
  }
  ### Fields in truth_data.jsonl:
  {
    
    "id": "<instance id>",
    "truthJudgments": [<number in [0,1]>],
    "truthMean": <number in [0,1]>,
    "truthMedian": <number in [0,1]>,
    "truthMode": <number in [0,1]>,
    "truthClass": "clickbait | no-clickbait"
  }

In [ ]:
# Mount data from drive
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from gensim.models import Word2Vec
import os 
import json
import nltk
import re


path2 = "/content/drive/MyDrive/IIITH- Folder/SMAI/SMAI_Project/clickbait17-validation-170630"

word2vec_model_file = path2 + '/' + 'word2vec_' + str(300) + '.model'

nltk_tokeniser = nltk.tokenize.TweetTokenizer()

def process_tweet(text):
    FLAGS = re.MULTILINE | re.DOTALL

    def hashtag(text):
        text = text.group()
        hashtag_body = text[1:]
        result = " ".join(["<hashtag>"] + re.split(r"(?=[A-Z])", hashtag_body, flags=FLAGS))
        return result

    def allcaps(text):
        text = text.group()
        return text.lower() + " <allcaps>"
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    text = re_sub(r"/"," / ")
    text = re_sub(r"@\w+", "<user>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"#\S+", hashtag)
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")

    text = re_sub(r"([A-Z]){2,}", allcaps)

    return text



def tokenise(text, with_process=True):
    if with_process:
        return nltk_tokeniser.tokenize(process_tweet(text).lower())
    else:
        # return nltk_tokeniser.tokenize(text)
        return tweet_ark_tokenize(text.lower())

def generate_embeddings(fp):
    sentences = []
    headline = []
    desc = []
    id = []
    # files = ["/MyDrive/clickbait17-validation-170630"]
    files = [path2]
    for each_fp in files:
        with open(os.path.join(each_fp, 'instances.jsonl'), 'rb') as f:
            for each_line in f:
                each_item = json.loads(each_line.decode('utf-8'))
                for each_sentence in each_item["id"]:
                    id.append(each_sentence)
                if each_item["targetTitle"]:
                    headline.append(tokenise(each_item["targetTitle"]))
                if each_item["targetDescription"]:
                    sentences.append(tokenise(each_item["targetDescription"]))
                for each_sentence in each_item["targetParagraphs"]:
                    desc.append(tokenise(each_sentence))
                for each_sentence in each_item["targetCaptions"]:
                    sentences.append(tokenise(each_sentence))
    word2vec_model_h = Word2Vec(headline) 
    word2vec_model_d = Word2Vec(desc) 
    # word2vec_model.wv.save_word2vec_format(os.path.join(fp, "s_clickbait.100.txt"), binary=False)
    return word2vec_model_h, word2vec_model_d, headline, desc


# def generate_embeddings(fp):
#     sentences = []
#     # files = ["/MyDrive/clickbait17-validation-170630"]
#     files = [path2]
#     for each_fp in files:
#         with open(os.path.join(each_fp, 'truth.jsonl'), 'rb') as f:
#             for each_line in f:
#                 each_item = json.loads(each_line.decode('utf-8'))
#                 for each_sentence in each_item["postText"]:
#                     sentences.append(tokenise(each_sentence))
#                 if each_item["targetTitle"]:
#                     sentences.append(tokenise(each_item["targetTitle"]))
#                 if each_item["targetDescription"]:
#                     sentences.append(tokenise(each_item["targetDescription"]))
#                 for each_sentence in each_item["targetParagraphs"]:
#                     sentences.append(tokenise(each_sentence))
#                 for each_sentence in each_item["targetCaptions"]:
#                     sentences.append(tokenise(each_sentence))
#     word2vec_model = Word2Vec(sentences) 
#     # word2vec_model.wv.save_word2vec_format(os.path.join(fp, "s_clickbait.100.txt"), binary=False)
#     return word2vec_model

word2vec_model_h, word2vec_model_d, headline, desc= generate_embeddings(path2)

# word2vec_model_h.wv.save_word2vec_format('h.txt',binary=False)
# word2vec_model_d.wv.save_word2vec_format('d.txt',binary=False)


# word2vec_model = generate_embeddings(path2)

# word2vec_model.wv.save(word2vec_model_file)

# print(word2vec_model['small'])



In [ ]:


# print(headline.shape)
# print(desc.shape)

Length of list using naive method is : 19538


In [ ]:
similar = word2vec_model.wv.most_similar('small')
print(similar)

[('large', 0.8136820793151855), ('tiny', 0.6736317873001099), ('smaller', 0.633418083190918), ('huge', 0.61185622215271), ('larger', 0.5739832520484924), ('different', 0.561408519744873), ('thin', 0.5400251150131226), ('remote', 0.5382583737373352), ('modest', 0.5186053514480591), ('buildings', 0.4981464743614197)]


## Get embeddings

In [ ]:
# print(headline[0], desc[0])
# print(headline[:2])
print(truth_means[:10])


[['‘', 'inexcusable', '’', 'failures', 'in', 'uk', '<allcaps>', '’', 's', 'response', 'to', 'modern', 'slavery', 'leaving', 'victims', 'destitute', 'while', 'abusers', 'go', 'free', ',', 'report', 'warns'], ['donald', 'trump', 'appoints', 'pro-life', 'advocate', 'as', 'assistant', 'secretary', 'of', 'hhs', '<allcaps>', 'for', 'public', 'affairs']]
[[0.13333333332], [1.0], [0.46666666664], [0.93333333332], [0.0], [0.06666666666], [0.33333333332], [0.06666666666], [0.3333333333], [0.13333333332]]


In [ ]:
truth_class = []

# print(type(truth_means[0][0]))

for i in range(len(truth_means)):
  
    if truth_means[i][0] > 0.5:
        truth_class.append(1)
    else:
        truth_class.append(0)

headline_train = headline[:14600]
headline_test = headline[14601:]

truth_class_train = truth_class[:14600]
truth_class_test = truth_class[14601:]

desc_train = desc[:14600]
desc_test = desc[14601:]


In [ ]:
print(len(headline_train))
print(len(desc_train))

def train_wordToVec(dataset, filename):
  size = 300
  window = 3
  min_count = 1
  workers = 3
  sg = 1 
  OUTPUT_FOLDER = "/content/drive/MyDrive/"
  word2vec_model_file1 = OUTPUT_FOLDER + filename + str(size) + '.model'
  start_time = time.time()
  stemmed_tokens1 = pd.Series(dataset).values
  # Train the Word2Vec Model
  w2v_model1 = Word2Vec(stemmed_tokens1, min_count = min_count, size = size, workers = workers, window = window, sg = sg)
  print("Time taken to train word2vec model: " + str(time.time() - start_time))
  w2v_model1.save(word2vec_model_file1)

  return stemmed_tokens1, word2vec_model_file1


stemmed_tokens1, word2vec_model_file1 = train_wordToVec(headline_train,"word2vec_1")
stemmed_tokens2, word2vec_model_file2 = train_wordToVec(desc_train,"word2vec_2" )


14600
14600
Time taken to train word2vec model: 10.01415228843689
Time taken to train word2vec model: 20.383568286895752


In [ ]:
w2v_model1= Word2Vec.load(word2vec_model_file1)
w2v_model2= Word2Vec.load(word2vec_model_file2)
w2v_model1.wv.save_word2vec_format('h2.txt',binary=False)
w2v_model2.wv.save_word2vec_format('d2.txt',binary=False)

In [ ]:
import os
def get_embed_index(filename):
  embed_index1={}
  f= open(os.path.join('',filename),encoding="utf-8")
  for line in f:
    values =line.split()
    word = values[0]
    coefs= np.asarray(values[1:])
    embed_index1[word]=coefs
  f.close()
  return embed_index1

embed_index1=get_embed_index('h2.txt')
embed_index2=get_embed_index('d2.txt')

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

def get_embed_matrix(stemmed_tokens, embed_index,type):
  tok_obj=Tokenizer()
  tok_obj.fit_on_texts(stemmed_tokens)
  sequences = tok_obj.texts_to_sequences(stemmed_tokens)

  word_index=tok_obj.word_index
  print('Found %s unique tokens.'% len(word_index))

  if type=='H':
    #max_length=max([len(s) for s in stemmed_tokens])
    max_length=25
  else:
    max_length=1500

  review_pad =pad_sequences(sequences,maxlen=max_length)

  print("Shape of review tensor:",review_pad.shape)
  num_words=len(word_index)+1
  embed_matrix=np.zeros((num_words,300))

  for word,i in word_index.items():
    if i > num_words:
      continue
    embedding_vector=embed_index.get(word)
    if embedding_vector is not None:
      embed_matrix[i]=embedding_vector
  return embed_matrix,num_words,max_length, review_pad


In [ ]:
embed_matrix1,num_words1,max_length1,tensor1=get_embed_matrix(stemmed_tokens1,embed_index1,'H')
import itertools
# my_list = [elem[0] for elem in your_dict.values()]
# print("\n", dict(itertools.islice(embed_index2.items(), 10)), "\n")

embed_index2.update({'.': embed_index2.get('.')[3:]})

# print(embed_index2.get('.')[1:])
# print(embed_index2.get('.')[0])

# print(embed_index2.get('.'))

embed_matrix2,num_words2,max_length2, tensor2=get_embed_matrix(stemmed_tokens2, embed_index2,'A')

Found 20298 unique tokens.
Shape of review tensor: (14600, 25)
Found 28945 unique tokens.
Shape of review tensor: (14600, 1500)


In [ ]:
print(tensor1.shape)
print(tensor2.shape)
print(embed_matrix1.shape)
print(embed_matrix2.shape)

(14600, 25)
(14600, 1500)
(20299, 300)
(28946, 300)


## Training the model

In [ ]:
from keras.initializers import Constant

embed1=Input(shape=(25,))
embed2=Input(shape=(1500,))
 
embedding_layer1 = Embedding( num_words1,300,embeddings_initializer=Constant(embed_matrix1),input_length=max_length1,trainable=False)(embed1)
embedding_layer2 = Embedding( num_words2,300,embeddings_initializer=Constant(embed_matrix2),input_length=max_length2,trainable=False)(embed2)
bigrulayer1=Bidirectional(GRU(units=250, dropout=0.3, recurrent_dropout=0.5))(embedding_layer1)
bigrulayer2=Bidirectional(GRU(units=250, dropout=0.3, recurrent_dropout=0.5))(embedding_layer2)
 
pin_part = Dense(64, activation='relu')(bigrulayer1)
sku_part = Dense(64, activation='relu')(bigrulayer2)

global_similarity=keras.layers.Dot(axes=1, normalize=True)([pin_part, sku_part])

probb= Dense(1, activation='sigmoid')(global_similarity)

model1 = Model(inputs=[embed1] + [embed2], outputs=probb)
model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1500)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 25, 300)      6089700     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1500, 300)    8683800     input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
inp_list = [tensor1, tensor2]
# out_list = train_body_stance_merged['Stance'][:1000]
out_list = np.array(truth_class_train, dtype=np.float64)

checkpoint_path = "'drive/MyDrive/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model1.fit(inp_list, out_list, batch_size = 64, epochs=2, callbacks=[cp_callback])

model1.save('drive/MyDrive/clickmodel.h5')

## Applying on test data

In [ ]:
stemmed_tokenst1, word2vec_model_filet1 = train_wordToVec(headline_test,"word2vec_t1")
stemmed_tokenst2, word2vec_model_filet2 = train_wordToVec(desc_test,"word2vec_t2" )

w2v_modelt1= Word2Vec.load(word2vec_model_filet1)
w2v_modelt2= Word2Vec.load(word2vec_model_filet1)

w2v_model1.wv.save_word2vec_format('myfilet1.txt',binary=False)
w2v_model2.wv.save_word2vec_format('myfilet2.txt',binary=False)

embed_indext1=get_embed_index('myfilet1.txt')
embed_indext2=get_embed_index('myfilet2.txt')

embed_matrixt1,num_wordst1,max_lengtht1,tensort1=get_embed_matrix(stemmed_tokenst1,embed_indext1,'H')


In [ ]:
embed_indext2.update({'.': embed_indext2.get('.')[3:]})
embed_matrixt2,num_wordst2,max_lengtht2, tensort2=get_embed_matrix(stemmed_tokenst2, embed_indext2,'A')

out_list2 = np.array(truth_class_test, dtype=np.float64)

score = model1.evaluate([tensort1,tensort2], out_list2, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

## Alt: Using glove for vectorisation 


In [ ]:
PAD = "<pad>"  
UNK = "<unk>"  
nltk_tokeniser = nltk.tokenize.TweetTokenizer()

EmbeddingSize = 100
glovepath = "/content/drive/MyDrive/SMAI_Project/glove.6B.zip (Unzipped Files)/"
filepath = "/content/drive/MyDrive/SMAI_Project/"

def WordEmbeddingLoader(fp, embedding_size):
    embedding = []
    vocab = []
    linenumber=0
    with open(fp, 'r', encoding='UTF-8') as f:
        for each_line in f:
          
            linenumber+=1            
            row = each_line.split(' ')
 
            if len(row) == 2:
                continue
            vocab.append(row[0])
            if len(row[1:]) != embedding_size:
                print (row[0])
                print (len(row[1:]))
            embedding.append(np.asarray(row[1:], dtype='float32'))
   
    word2id = dict(zip(vocab, range(2, len(vocab)+2)))
    word2id[PAD] = 0
    word2id[UNK] = 1

    extra_embedding = [np.zeros(embedding_size), np.random.uniform(-0.1, 0.1, embedding_size)]
    embedding = np.append(extra_embedding, embedding, 0)
    return word2id, embedding,vocab


def data_reader(fps, word2id=None, y_len=1):
    ids = []
    post_texts = []
    post_text_lens = []
    truth_means = []
    truth_classes = []
    id2truth_class = {}
    id2truth_mean = {}
    for fp in fps:
        if y_len:
            with open(os.path.join(fp, 'truth.jsonl'), 'rb') as fin:
                for each_line in fin:
                    each_item = json.loads(each_line.decode('utf-8'))
                    if y_len == 4:
                        each_label = [0, 0, 0, 0]
                        
                        for each_key, each_value in Counter(each_item["truthJudgments"]).items():
                            each_label[int(each_key//0.3)] = float(each_value)/5
                        id2truth_class[each_item["id"]] = each_label
                        if each_item["truthClass"] != "clickbait":
                            assert each_label[0]+each_label[1] > each_label[2]+each_label[3]
                        else:
                            assert each_label[0]+each_label[1] < each_label[2]+each_label[3]
                    if y_len == 2:
                        if each_item["truthClass"] == "clickbait":
                            id2truth_class[each_item["id"]] = [1, 0]
                        else:
                            id2truth_class[each_item["id"]] = [0, 1]
                    if y_len == 1:
                        if each_item["truthClass"] == "clickbait":
                            id2truth_class[each_item["id"]] = [1]
                        else:
                            id2truth_class[each_item["id"]] = [0]
                    id2truth_mean[each_item["id"]] = [float(each_item["truthMean"])]
        
        with open(os.path.join(fp, 'instances.jsonl'), 'rb') as fin:
            for each_line in fin:
                each_item = json.loads(each_line.decode('utf-8'))
                if each_item["id"] not in id2truth_class and y_len:
                    num += 1
                    continue
                ids.append(each_item["id"])
                each_post_text = " ".join(each_item["postText"])
                each_target_description = each_item["targetTitle"]
                if y_len:
                    truth_means.append(id2truth_mean[each_item["id"]])
                    truth_classes.append(id2truth_class[each_item["id"]])
                if word2id:
                    if (each_post_text+" ").isspace():
                        #the id of <unk>
                        post_texts.append([0])
                        post_text_lens.append(1)
                    else:
                        each_post_tokens = tokeniser(each_post_text)
                        post_texts.append([word2id.get(each_token, 1) for each_token in each_post_tokens])
                        post_text_lens.append(len(each_post_tokens))
                else:
                    post_texts.append([each_post_text])
    return ids, post_texts, truth_classes, post_text_lens, truth_means


def Sequence_pader(sequences, maxlen):
    if maxlen <= 0:
        return sequences
    shape = (len(sequences), maxlen)
    padded_sequences = np.full(shape, 0)
    for i, each_sequence in enumerate(sequences):
        if len(each_sequence) > maxlen:
            padded_sequences[i] = each_sequence[:maxlen]
        else:
            padded_sequences[i, :len(each_sequence)] = each_sequence
    return padded_sequences



def tokeniser(text, with_process=True):
    if with_process:
        return nltk_tokeniser.tokenize(tweet_processor(text).lower())
        print("ar")
    else:
        return simpleTokenize(squeezeWhitespace(text.lower()))
        print("br")

def tweet_processor(text):
    FLAGS = re.MULTILINE | re.DOTALL
    
    def megasplit(pattern, string):
        splits = list((m.start(), m.end()) for m in re.finditer(pattern, string))
        starts = [0] + [i[1] for i in splits]
        ends = [i[0] for i in splits] + [len(string)]
        return [string[start:end] for start, end in zip(starts, ends)]
    
    def hashtag(text):
        text = text.group()
        hashtag_body = text[1:]
        #print(hashtag_body)
        
        #result = " ".join(["<hashtag>"] + re.split(r"(?=[A-Z])", hashtag_body, flags=FLAGS))
        result = " ".join(["<hashtag>"] + megasplit(r"(?=[A-Z])", hashtag_body))
        return result

    def allcaps(text):
        text = text.group()
        return text.lower() + " <allcaps>"
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    text = re_sub(r"/"," / ")
    text = re_sub(r"@\w+", "<user>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"#\S+", hashtag)
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")
    text = re_sub(r"([A-Z]){2,}", allcaps)

    return text



np.random.seed(81)
word2id, embedding_matrix,vocab = WordEmbeddingLoader(fp=os.path.join(glovepath, "glove.6B."+str(EmbeddingSize)+"d.txt"), embedding_size=EmbeddingSize)

# import itertools
# print("start")
# print("\n", dict(itertools.islice(test_dict.items(), 10)), type(word2id), "\n")
# print(embedding_matrix[:10], type(embedding_matrix), "\n")
# print(vocab[:10], type(vocab), "\n")
# print("end")

with open(os.path.join(filepath, 'word2id.json'), 'w') as fout:
    json.dump(word2id, fp=fout)
    

ids, post_texts, truth_classes, post_text_lens, truth_means = data_reader(word2id=word2id, fps=[os.path.join(filepath, 'clickbait17-validation-170630')], y_len=4)

# print(type(ids), type(post_texts), type(post_text_lens), type(target_descriptions))
# print(ids[0], post_texts[0], post_text_lens[0], truth_means[0], target_descriptions[0], target_description_lens[0])
# 858462320779026433 [2048, 1, 3073, 1536, 1155, 6, 1196, 8177, 1222, 1218, 25254, 112, 26971, 244, 417] 15 [0.13333333332] [] 0
# print("\n", len(ids), len(post_texts), len(post_text_lens), len(target_descriptions), "\n") 
# 19538  14600
ids_train = ids [:14600]
ids_test = ids [14601:]
post_texts_train = post_texts[:14600]
post_texts_test = post_texts[14601:]
truth_classes_train = truth_classes[:14600]
truth_classes_test = truth_classes[14601:]
post_text_lens_train = post_text_lens[:14600]
post_text_lens_test = post_text_lens[14601:]
truth_means_train = truth_means[:14600]
truth_means_test = truth_means[14601:]

# print(ids_train[0])

# print("\n", len(ids1), len(ids2), "\n")

post_texts_train = np.array(post_texts_train)
truth_classes_train = np.array(truth_classes_train)
post_text_lens_train = np.array(post_text_lens_train)
truth_means_train = np.array(truth_means_train)
shuffle_indices = np.random.permutation(np.arange(len(post_texts_train)))
post_texts_train = post_texts_train[shuffle_indices]
truth_classes_train = truth_classes_train[shuffle_indices]
post_text_lens_train = post_text_lens_train[shuffle_indices]
truth_means_train = truth_means_train[shuffle_indices]
max_post_text_len = max(post_text_lens_train)
# print(max_post_text_len)

post_texts_train = Sequence_pader(post_texts_train, max_post_text_len)

post_texts_test = np.array(post_texts_test)
truth_classes_test = np.array(truth_classes_test)
post_text_lens_test = [each_len if each_len <= max_post_text_len else max_post_text_len for each_len in post_text_lens_test]
post_text_lens_test = np.array(post_text_lens_test)
truth_means_test = np.array(truth_means_test)
truth_means_test = np.ravel(truth_means_test).astype(np.float32)
post_texts_test = Sequence_pader(post_texts_test, max_post_text_len)

max_features = len(word2id.keys())
maxlen = max_post_text_len
EmbeddingSize = 100
dropout_embedding = 0.2
X_train = post_texts_train
y_train = truth_means_train
X_test = post_texts_test
y_test = truth_means_test


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:201: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


## Glove Model

In [ ]:
# build the keras LSTM model
model = Sequential()

model.add(Embedding(input_dim = max_features, output_dim = EmbeddingSize, weights = [embedding_matrix], input_length = maxlen ,
                    trainable = False))
model.add(Dropout(dropout_embedding))
model.add(Bidirectional(GRU(512, dropout=0.2, recurrent_dropout=0.5)))  
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='mse', optimizer='rmsprop')

batch_size = 64

earlystop_cb = keras.callbacks.EarlyStopping(monitor='mse', patience=7, verbose=1, mode='auto')

model.fit(X_train, y_train, batch_size=batch_size, epochs=20, validation_split=0.1, callbacks=[earlystop_cb])

petruth_means = model.predict(X_test)
tetruthClass = []
petruthClass = []


for i in range(len(truth_means_test)):
  
    if petruth_means[i] > 0.5:
        petruthClass.append(1)
    else:
        petruthClass.append(0)
    
    if truth_means_test[i] > 0.5:
        tetruthClass.append(1)
    else:
        tetruthClass.append(0)


mse = metrics.mean_squared_error(truth_means_test, petruth_means)
print('Mean Squared Error = '+str(mse))

accuracy = metrics.accuracy_score(tetruthClass,petruthClass)
print('accuracy = '+str(accuracy))

precision = metrics.precision_score(tetruthClass,petruthClass)
print('precision_score = '+str(precision))

recall = metrics.recall_score(tetruthClass,petruthClass)
print('recall_score = '+str(recall))

f1 = metrics.f1_score(tetruthClass,petruthClass)
print('f1_score = '+str(f1))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:201: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/20
206/206 [==============================] - 59s 253ms/step - loss: 0.0517 - val_loss: 0.0377
Epoch 2/20
206/206 [==============================] - 51s 248ms/step - loss: 0.0373 - val_loss: 0.0325
Epoch 3/20
206/206 [==============================] - 51s 247ms/step - loss: 0.0347 - val_loss: 0.0358
Epoch 4/20
206/206 [==============================] - 52s 251ms/step - loss: 0.0329 - val_loss: 0.0302
Epoch 5/20
206/206 [==============================] - 51s 248ms/step - loss: 0.0322 - val_loss: 0.0321
Epoch 6/20
206/206 [==============================] - 51s 250ms/step - loss: 0.0314 - val_loss: 0.0280
Epoch 7/20
206/206 [==============================] - 51s 248ms/step - loss: 0.0298 - val_loss: 0.0311
Epoch 8/20
206/206 [==============================] - 52s 251ms/step - loss: 0.0299 - val_loss: 0.0287
Epoch 9/20
206/206 [==============================] - 51s 248ms/step - loss: 0.0276 - val_loss: 0.0312
Epoch 10/20
206/206 [==============================] - 51s 246ms/step - l

In [ ]:
print(truth_means[:10])

[[0.13333333332], [1.0], [0.46666666664], [0.93333333332], [0.0], [0.06666666666], [0.33333333332], [0.06666666666], [0.3333333333], [0.13333333332]]


In [ ]:
from keras.models import load_model

model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# returns a compiled model identical to the previous one
# model = load_model('my_model.h5')

# import pickle
# import joblib
  
# # save the model to disk
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))

# joblib.dump(model, filename) 
# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity

# from sklearn.feature_extraction.text import TfidfVectorizer

# baseFolderPath = path2

# fileNames, filePathList = returnListOfFilePaths(baseFolderPath)

# rawContentDict = create_docContentDict(filePathList)

# # calculate tfidf
# tfidf = TfidfVectorizer(tokenizer=processData, 'english') #stop_words
# tfs = tfidf.fit_transform(rawContentDict.values())


# # TODO: modify this to build matrix then print from matrix form
# def calc_and_print_CosineSimilarity_for_all(tfs, fileNames):
#     #print(cosine_similarity(tfs[0], tfs[1]))
#     numFiles = len(fileNames)
#     names = []
#     print('                   ', end="")    #formatting
#     for i in range(numFiles):
#         if i == 0:
#             for k in range(numFiles):
#                 print(fileNames[k], end='   ')
#             print()

#         print(fileNames[i], end='   ')
#         for n in range(numFiles):
#             #print(fileNames[n], end='\t')
#             matrixValue = cosine_similarity(tfs[i], tfs[n])
#             numValue = matrixValue[0][0]
#             #print(numValue, end='\t')
#             names.append(fileNames[n])
#             print(" {0:.8f}".format(numValue), end='         ')
#             #(cosine_similarity(tfs[i], tfs[n]))[0][0]

#         print()

In [ ]:
# class MyBasicAttentiveBiGRU(models.Model):

#     def __init__(self, vocab_size: int, embed_dim: int, hidden_size: int = 128, training: bool = False):
#         super(MyBasicAttentiveBiGRU, self).__init__()

#         self.num_classes = len(ID_TO_CLASS)

#         self.decoder = layers.Dense(units=self.num_classes)
#         self.omegas = tf.Variable(tf.random.normal((hidden_size*2, 1)))

#         ### TODO(Students) START
#         self.embeddings = tf.Variable(tf.random.normal((vocab_size, embed_dim)), trainable = training) 
        
#         self._forward_layer = layers.GRU(hidden_size, return_sequences=True)
#         self._backward_layer = layers.GRU(hidden_size, return_sequences=True, go_backwards=True)
#         self._bidirectional_layer = layers.Bidirectional(self._forward_layer, backward_layer=self._backward_layer, merge_mode = 'concat')
#         ### TODO(Students) END

#     def attn(self, rnn_outputs):
#         ### TODO(Students) START

#         # The following lines are based on the equation 9-12 by Zhou et al.
        
#         # Equation 9: M = tanh(H)
#         M = tf.tanh(rnn_outputs) # 10, 5, 256

#         # Equation 10: \alpha = softmax(w^T.M)
#         alpha = tf.nn.softmax(tf.tensordot(M, self.omegas, axes=1))  # 10, 5, 1

#         # Equation 11: H.\alpha^T
#         r = tf.reduce_sum(rnn_outputs * alpha, axis=1)

#         # Equation 12: tanh(r)
#         output = tf.tanh(r)

#         ### TODO(Students) END

#         return output

#     def call(self, inputs, pos_inputs, training):
        
#         word_embed = tf.nn.embedding_lookup(self.embeddings, inputs) # 10, 5, 100
#         pos_embed = tf.nn.embedding_lookup(self.embeddings, pos_inputs) # 10, 5, 100
        
#         ### TODO(Students) START

#         # First, we will concatenate the word and POS embeddings along the 3rd axis
#         input_for_bidirectional = tf.concat([word_embed, pos_embed], axis=2) # 10, 5, 200
        
#         # input_for_bidirectional = word_embed # This is for experiments, comment the above line

#         # Create a mask for the sequence as it is padded with 0
#         mask = tf.cast(tf.greater(inputs, 0), tf.float32)

#         # Pass the word (+ POS) embeddings to bidirectional GRU layer
#         hidden_outputs = self._bidirectional_layer(input_for_bidirectional, mask = mask) # 10, 5, 256

#         # Apply attention to the hidden state outputs from forward and backward layer of biGRU
#         attn = self.attn(hidden_outputs)

#         # Apply the dense layer to obtain logits
#         logits = self.decoder(attn)
#         ### TODO(Students) END

#         # batch_size, num_classes
#         return {'logits': logits}



---

